##NER 모델링 및 학습

In [ ]:
!git clone https://github.com/ukairia777/tensorflow-bert-ner.git

Cloning into 'tensorflow-bert-ner'...
remote: Enumerating objects: 1531, done.
remote: Counting objects: 100% (1531/1531), done.
remote: Compressing objects: 100% (1528/1528), done.
remote: Total 1531 (delta 33), reused 1443 (delta 2), pack-reused 0
Receiving objects: 100% (1531/1531), 6.23 MiB | 10.49 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [ ]:
%ls

sample_data/  tensorflow-bert-ner/


In [ ]:
%cd tensorflow-bert-ner/

/content/tensorflow-bert-ner


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.0 MB/s eta 0:00:00


In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=9971bf89ea46ce277a1a9f4724305503026d31a78bde1bcf92b986278d1fbbf4
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [ ]:
import glob, re
import numpy as np
import codecs
import os
import json
from tqdm import tqdm
import tensorflow as tf
from seqeval.metrics import f1_score, classification_report
from transformers import shape_list, BertTokenizer, TFBertModel

In [ ]:
train_dataset = glob.glob('data/train_data/*.txt')
test_dataset = glob.glob('data/validation_data/*.txt')

In [ ]:
len(train_dataset)

1425

In [ ]:
len(test_dataset)

2

In [ ]:
def read_ner_data(dataset):

  tagged_sentences = []
  sentence = []

  for file_name in tqdm(dataset):
    f = open(file_name, 'r', encoding='utf-8-sig')
    for line in f:
      if len(line)==0 or line[:2]=="##" or line[0]=="\n":
          if len(sentence) > 0:
              tagged_sentences.append(sentence)
              sentence = []
          continue
      splits = line.split('\t') # 공백을 기준으로 속성을 구분한다.
      splits[-1] = re.sub(r'\n', '', splits[-1]) # 줄바꿈 표시 \n을 제거한다.
      word = splits[0]
      sentence.append([word, splits[-1]]) # 단어와 개체명 태깅만 기록한다.

  return tagged_sentences

In [ ]:
train_tagged_sentences = read_ner_data(train_dataset)
test_tagged_sentences = read_ner_data(test_dataset)

100%|██████████| 2/2 [00:00<00:00, 28.13it/s]


In [ ]:
print(len(train_tagged_sentences))
print(len(test_tagged_sentences))

23033
930


In [ ]:
train_tagged_sentences[0]

[['전국', 'O'],
 ['_', 'O'],
 ['수', 'O'],
 ['의', 'O'],
 ['_', 'O'],
 ['및', 'O'],
 ['_', 'O'],
 ['축산', 'O'],
 [',', 'O'],
 ['_', 'O'],
 ['말', 'O'],
 ['_', 'O'],
 ['관련', 'O'],
 ['_', 'O'],
 ['학과', 'O'],
 ['_', 'O'],
 ['재학생', 'O'],
 ['(', 'O'],
 ['휴학', 'O'],
 ['생', 'O'],
 ['_', 'O'],
 ['포함', 'O'],
 [')', 'O'],
 ['이', 'O'],
 ['면', 'O'],
 ['_', 'O'],
 ['누구', 'O'],
 ['나', 'O'],
 ['_', 'O'],
 ['신청', 'O'],
 ['가능', 'O'],
 ['하며', 'O'],
 [',', 'O'],
 ['_', 'O'],
 ['원서', 'O'],
 ['접수', 'O'],
 ['는', 'O'],
 ['_', 'O'],
 ['6', 'B-DAT'],
 ['월', 'I-DAT'],
 ['_', 'I-DAT'],
 ['25', 'I-DAT'],
 ['일', 'I-DAT'],
 ['부터', 'O'],
 ['_', 'O'],
 ['7', 'B-DAT'],
 ['월', 'I-DAT'],
 ['_', 'I-DAT'],
 ['9', 'I-DAT'],
 ['일', 'I-DAT'],
 ['까지', 'O'],
 ['_', 'O'],
 ['한국', 'B-ORG'],
 ['마사회', 'I-ORG'],
 ['_', 'O'],
 ['렛츠', 'O'],
 ['런', 'O'],
 ['_', 'O'],
 ['호스', 'O'],
 ['피', 'O'],
 ['아', 'O'],
 ['_', 'O'],
 ['홈페이지', 'O'],
 ['(', 'O'],
 ['www', 'B-POH'],
 ['.', 'I-POH'],
 ['horsepia', 'I-POH'],
 ['.', 'I-POH'],
 ['com', 'I-POH'],

In [ ]:
def split_sentence_and_label(tagged_sentences):
  index = 0
  sentences = []
  ner_tags = []

  for tagged_sentence in tqdm(tagged_sentences):
    sentence = []
    ner_tag = []
    for word, label in tagged_sentence:
      if word == '_' or word == '\xad':
        continue
      sentence.append(word)
      ner_tag.append(label)

    assert len(sentence) == len(ner_tag), "Error with input length {} vs {}".format(len(sentence), len(ner_tag))

    sentences.append(sentence)
    ner_tags.append(ner_tag)
  
  return sentences, ner_tags

In [ ]:
train_sentences, train_labels = split_sentence_and_label(train_tagged_sentences)
test_sentences, test_labels = split_sentence_and_label(test_tagged_sentences)

100%|██████████| 930/930 [00:00<00:00, 29789.09it/s]


In [ ]:
print(train_sentences[:3])
print(train_labels[:3])

[['전국', '수', '의', '및', '축산', ',', '말', '관련', '학과', '재학생', '(', '휴학', '생', '포함', ')', '이', '면', '누구', '나', '신청', '가능', '하며', ',', '원서', '접수', '는', '6', '월', '25', '일', '부터', '7', '월', '9', '일', '까지', '한국', '마사회', '렛츠', '런', '호스', '피', '아', '홈페이지', '(', 'www', '.', 'horsepia', '.', 'com', ')', '를', '통해서', '가능하', '다', '.'], ['팀', '별', '로', '30', ',', '31', '경기', '씩', '치른', '22', '일', '현재', '1', '∼', '3', '위', '인', '전북', ',', '수원', ',', '포항', '은', '상위', '리그', '행', '을', '확정', '했', '다', '.'], ['10', '일', '오전', '3', '시', '20', '분', '경', '경남', '통영시', '산양읍', '의', '한', '2', '층', '주택', '에서', '김', '모', '(', '67', ')', '황모', '씨', '(', '66', '·', '여', ')', '부부', '가', '동네', '주민', '설모', '씨', '(', '22', ')', '가', '휘두른', '흉기', '에', '찔려', '숨지', '는', '사고', '가', '발생', '했', '다', '.']]
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DAT', 'I-DAT', 'I-DAT', 'I-DAT', 'O', 'B-DAT', 'I-DAT', 'I-DAT', 'I-DAT', 'O', 'B-ORG', 'I-

In [ ]:
print(test_sentences[:3])
print(test_labels[:3])

[['박찬호', '는', '8', '일', '(', '한국', '시간', ')', '애리조나', '다이아몬드백스', '와', '의', '원정', '경기', '에서', '다저스', '가', '1', '-', '4', '로', '뒤지', '던', '5', '회', '말', '선발', '에스테반', '로아', '이사', '로부터', '볼', '을', '건네', '받', '았', '다', '.'], ['하지만', '5', '번', '마크', '레이놀스', '에게', '우측', '담장', '을', '넘기', '는', '솔로', '포', '를', '얻어맞', '으면서', '박찬호', '의', '상승세', '는', '무너', '졌', '다', '.'], ['-', '1', '번', '크리스', '영', '1', '구', '.']]
[['B-PER', 'O', 'B-DAT', 'I-DAT', 'O', 'B-LOC', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'B-NOH', 'I-NOH', 'I-NOH', 'O', 'O', 'O', 'B-NOH', 'I-NOH', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-NOH', 'I-NOH', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-NOH', 'I-NOH', 'B-PER', 'O', 'B-NOH', 'O', 'O']]


In [ ]:
labels = set(tag for train_label in train_labels for tag in train_label)

In [ ]:
print(labels)

{'B-PNT', 'B-DAT', 'B-ORG', 'O', 'I-ORG', 'I-PER', 'I-LOC', 'I-POH', 'B-TIM', 'B-LOC', 'I-TIM', 'B-MNY', 'I-NOH', 'I-DUR', 'B-NOH', 'I-DAT', 'B-POH', 'B-PER', 'B-DUR', 'I-MNY', 'I-PNT'}


In [ ]:
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

In [ ]:
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

In [ ]:
print(tag_to_index)
print(index_to_tag)

{'B-PNT': 0, 'B-DAT': 1, 'B-ORG': 2, 'O': 3, 'I-ORG': 4, 'I-PER': 5, 'I-LOC': 6, 'I-POH': 7, 'B-TIM': 8, 'B-LOC': 9, 'I-TIM': 10, 'B-MNY': 11, 'I-NOH': 12, 'I-DUR': 13, 'B-NOH': 14, 'I-DAT': 15, 'B-POH': 16, 'B-PER': 17, 'B-DUR': 18, 'I-MNY': 19, 'I-PNT': 20}
{0: 'B-PNT', 1: 'B-DAT', 2: 'B-ORG', 3: 'O', 4: 'I-ORG', 5: 'I-PER', 6: 'I-LOC', 7: 'I-POH', 8: 'B-TIM', 9: 'B-LOC', 10: 'I-TIM', 11: 'B-MNY', 12: 'I-NOH', 13: 'I-DUR', 14: 'B-NOH', 15: 'I-DAT', 16: 'B-POH', 17: 'B-PER', 18: 'B-DUR', 19: 'I-MNY', 20: 'I-PNT'}


In [ ]:
tag_size = len(tag_to_index)
print('개체명 태깅 정보의 개수 :',tag_size)

개체명 태깅 정보의 개수 : 21


In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):

    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        tokens = []
        labels_ids = []
        for one_word, label_token in zip(example, label):
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        labels_ids += [pad_token_id_for_label]

        tokens = [cls_token] + tokens
        labels_ids = [pad_token_id_for_label] + labels_ids


        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label = labels_ids + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
X_train, y_train = convert_examples_to_features(train_sentences, train_labels, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 23033/23033 [00:40<00:00, 574.90it/s]


In [ ]:
X_test, y_test = convert_examples_to_features(test_sentences, test_labels, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 930/930 [00:00<00:00, 1220.34it/s]


In [ ]:
class TFBertForTokenClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFBertForTokenClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        all_output = outputs[0]
        prediction = self.classifier(all_output)

        return prediction

In [ ]:
def compute_loss(labels, logits):

  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
  active_loss = tf.reshape(labels, (-1,)) != -100
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)

  return loss_fn(labels, reduced_logits)

In [ ]:
model = TFBertForTokenClassification("klue/bert-base", num_labels=tag_size)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=compute_loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'bert.embeddings.position_ids', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test

    def sequences_to_tags(self, label_ids, pred_ids):
      label_list = []
      pred_list = []

      for i in range(0, len(label_ids)):
        label_tag = []
        pred_tag = []

        for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
          if label_index != -100:
            label_tag.append(index_to_tag[label_index])
            pred_tag.append(index_to_tag[pred_index])
        
        label_list.append(label_tag)
        pred_list.append(pred_tag)

      return label_list, pred_list

    def on_epoch_end(self, epoch, logs={}):

      y_predicted = self.model.predict(self.X_test)
      y_predicted = np.argmax(y_predicted, axis = 2)

      label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

      score = f1_score(label_list, pred_list)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(label_list, pred_list))

In [ ]:
f1_score_report = F1score(X_test, y_test)

In [ ]:
model.fit(
    X_train, y_train, epochs=3, batch_size=32,
    callbacks = [f1_score_report]
)

Epoch 1/3


30/30 [==============================] - 12s 269ms/step
 - f1: 86.22
              precision    recall  f1-score   support

         DAT       0.94      0.97      0.95       182
         DUR       0.63      0.54      0.58        50
         LOC       0.83      0.81      0.82       205
         MNY       0.83      1.00      0.91        20
         NOH       0.84      0.87      0.86      1007
         ORG       0.83      0.92      0.87       795
         PER       0.92      0.96      0.94       853
         PNT       0.81      0.80      0.81        60
         POH       0.55      0.59      0.57       214
         TIM       0.94      0.89      0.92        19

   micro avg       0.84      0.88      0.86      3405
   macro avg       0.81      0.84      0.82      3405
weighted avg       0.84      0.88      0.86      3405

720/720 [==============================] - 587s 790ms/step - loss: 0.1644
Epoch 2/3
30/30 [==============================] - 8s 269ms/step
 - f1: 86.69
              precis

##Mecab-ko

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/ner

/content/drive/MyDrive/ner


In [ ]:
# Only for korean dataset
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/drive/MyDrive/ner/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 44.4 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-02-09 06:12:44--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::22c0:3470, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-k

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()

In [ ]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]

        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask


        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [ ]:
def ner_prediction(examples, max_seq_len, tokenizer, lang='ko'):
  
  if lang == 'ko':
    examples = [mecab.morphs(sent) for sent in examples]
  else:
    examples = [sent.split() for sent in examples]

  X_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len=128, tokenizer=tokenizer)
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted, axis = 2)

  pred_list = []
  result_list = []

  for i in range(0, len(label_masks)):
    pred_tag = []
    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])

    pred_list.append(pred_tag)

  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)

  return result_list

##맞춤법 검사

In [ ]:
import requests
import json
import time
import sys
from collections import OrderedDict
import xml.etree.ElementTree as ET

In [ ]:
from collections import namedtuple

# 조사와 어미도 단어로 처리함. 마땅한 영단어가 생각이 안 나서..
_checked = namedtuple('Checked',
    ['result', 'original', 'checked', 'errors', 'words', 'time'])


class Checked(_checked):
    def __new__(cls, result=False, original='', checked='', errors=0, words=[], time=0.0):
        return super(Checked, cls).__new__(
            cls, result, original, checked, errors, words, time)

    def as_dict(self):
        d = {
            'result': self.result,
            'original': self.original,
            'checked': self.checked,
            'errors': self.errors,
            'words': self.words,
            'time': self.time,
        }
        return d

    def only_checked(self):
        return self.checked

In [ ]:
base_url = 'https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn'


class CheckResult:
    PASSED = 0
    WRONG_SPELLING = 1
    WRONG_SPACING = 2
    AMBIGUOUS = 3
    STATISTICAL_CORRECTION = 4

In [ ]:

_agent = requests.Session()
PY3 = sys.version_info[0] == 3


def _remove_tags(text):
    text = u'<content>{}</content>'.format(text).replace('<br>','')
    if not PY3:
        text = text.encode('utf-8')

    result = ''.join(ET.fromstring(text).itertext())

    return result


def check(text):
    """
    매개변수로 입력받은 한글 문장의 맞춤법을 체크합니다.
    """
    if isinstance(text, list):
        result = []
        for item in text:
            checked = check(item)
            result.append(checked)
        return result

    # 최대 500자까지 가능.
    if len(text) > 500:
        return Checked(result=False)

    payload = {
        '_callback': 'window.__jindo2_callback._spellingCheck_0',
        'q': text
    }

    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
        'referer': 'https://search.naver.com/',
    }

    start_time = time.time()
    r = _agent.get(base_url, params=payload, headers=headers)
    passed_time = time.time() - start_time

    r = r.text[42:-2]

    data = json.loads(r)
    html = data['message']['result']['html']
    result = {
        'result': True,
        'original': text,
        'checked': _remove_tags(html),
        'errors': data['message']['result']['errata_count'],
        'time': passed_time,
        'words': OrderedDict(),
    }

    # 띄어쓰기로 구분하기 위해 태그는 일단 보기 쉽게 바꿔둠.
    # ElementTree의 iter()를 써서 더 좋게 할 수 있는 방법이 있지만
    # 이 짧은 코드에 굳이 그렇게 할 필요성이 없으므로 일단 문자열을 치환하는 방법으로 작성.
    html = html.replace('<span class=\'green_text\'>', '<green>') \
               .replace('<span class=\'red_text\'>', '<red>') \
               .replace('<span class=\'purple_text\'>', '<purple>') \
               .replace('<span class=\'blue_text\'>', '<blue>') \
               .replace('</span>', '<end>')
    items = html.split(' ')
    words = []
    tmp = ''
    for word in items:
        if tmp == '' and word[:1] == '<':
            pos = word.find('>') + 1
            tmp = word[:pos]
        elif tmp != '':
            word = u'{}{}'.format(tmp, word)
        
        if word[-5:] == '<end>':
            word = word.replace('<end>', '')
            tmp = ''

        words.append(word)

    for word in words:
        check_result = CheckResult.PASSED
        if word[:5] == '<red>':
            check_result = CheckResult.WRONG_SPELLING
            word = word.replace('<red>', '')
        elif word[:7] == '<green>':
            check_result = CheckResult.WRONG_SPACING
            word = word.replace('<green>', '')
        elif word[:8] == '<purple>':
            check_result = CheckResult.AMBIGUOUS
            word = word.replace('<purple>', '')
        elif word[:6] == '<blue>':
            check_result = CheckResult.STATISTICAL_CORRECTION
            word = word.replace('<blue>', '')
        result['words'][word] = check_result

    result = Checked(**result)

    return result

In [ ]:
def text_checker(error_text):
  if len(error_text) >= 500:
    ready_list = []
    check_list=[]
    while (len(error_text) >= 500):
      temp_str = error_text[:500]
      last_space = temp_str.rfind(' ')
      temp_str = error_text[0:last_space]
      ready_list.append(temp_str)

      error_text = error_text[last_space:]
    ready_list.append(error_text)

    for i in range(len(ready_list)):
      check_list.append(check(ready_list[i]).checked)
    
    sum_check_list = ' '.join(check_list)
    return sum_check_list

  else:
    return check(error_text).checked

In [ ]:
error_text = "사장님 2월9일김강현과 고려대학교의 하나스퀘어에서 11시 30분에 회의를 진행 할 예정입니다."

In [ ]:
error_text = "2월9일김강현과 고려대학교의 과학도서관에서 11시 30분에 회의를 진행 할 예정입니다."

In [ ]:
error_text = "사장님 2월 9일 고려대학교의 황현수와 11시 30분에 하나스퀘어에서 회의를 진행 할 예정입니다."

In [ ]:
error_text = "여기 안암역압에 사람이 쓰러져있어요 교려대 학생같은데빨리 와주세요"

In [ ]:
sent1=text_checker(error_text)
sent1

'사장님 2월 9일 고려대학교의 황현수와 11시 30분에 하나스퀘어에서 회의를 진행할 예정입니다.'

NER 개체명 인식

In [ ]:
# sent1 = '보이그룹들의 글로벌 행보가 뚜렷했던 과거와 달리 걸그룹들의 글로벌 인기도 늘어나면서 K팝의 인기가 다변화되고 있다.'
# sent2 = '최근 글로벌 K팝 시장에서는 큰 변화가 생기고 있다.'
# sent3 = '그룹 블랙핑크를 필두로 하는 걸그룹들의 활약이 두드러지고 있는 것'
# sent4 = '특히 신인 걸그룹 뉴진스, 르세라핌, 아이브를 비롯해 트와이스, 있지 등의 글로벌 활약이 뚜렷해지면서 보이그룹 중심이었던 글로벌 K팝 시장도 큰 변화를 보이고 있다.'
# sent5 = '가장 먼저 글로벌 음원 시장에서 큰 족적을 남긴 건 블랙핑크였다.'

In [ ]:
def ner_def(sent1):
  test_samples = [sent1]
  result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer, lang='ko')
  return result_list

In [ ]:
result_list = ner_def(sent1)
result_list

100%|██████████| 1/1 [00:00<00:00, 819.20it/s]

1/1 [==============================] - 0s 57ms/step


[[('사장', 'O'),
  ('님', 'O'),
  ('2', 'B-DAT'),
  ('월', 'I-DAT'),
  ('9', 'I-DAT'),
  ('일', 'I-DAT'),
  ('고려', 'B-ORG'),
  ('대학교', 'I-ORG'),
  ('의', 'O'),
  ('황현수', 'B-PER'),
  ('와', 'O'),
  ('11', 'B-TIM'),
  ('시', 'I-TIM'),
  ('30', 'I-TIM'),
  ('분', 'I-TIM'),
  ('에', 'O'),
  ('하나', 'B-LOC'),
  ('스퀘어', 'I-LOC'),
  ('에서', 'O'),
  ('회의', 'O'),
  ('를', 'O'),
  ('진행할', 'O'),
  ('예정', 'O'),
  ('입니다', 'O'),
  ('.', 'O')]]

##결과요약

In [ ]:
def keyword(ner_list):
    
    dat_list = []
    per_list = []
    org_list = []
    tim_list = []
    loc_list = []
    
    for i in range(len(ner_list[0])):
        word = ner_list[0][i][0]
        tag = ner_list[0][i][1]
        
        if len(tag) == 5:
            if tag[2:] == 'DAT':
                dat_list.append(word)

        if len(tag) == 5:
            if tag[2:] == 'ORG':
                org_list.append(word)
                
        if len(tag) == 5:
            if tag[2:] == 'PER':
                per_list.append(word)
                
        if len(tag) == 5:
            if tag[2:] == 'TIM':
                tim_list.append(word)
                
        if len(tag) == 5:
            if tag[2:] == 'LOC':
                loc_list.append(word)

    dat = ''
    for i in dat_list:
        dat += i

    org = ''
    for i in org_list:
        org += i

    per = ''
    for i in per_list:
        per += i

    tim = ''
    for i in tim_list:
        tim += i
        
    loc = ''
    for i in loc_list:
        loc += i
        
    return print(f"기관 : {org} \n장소 : {loc} \n날짜 : {dat} \n시간 : {tim}\n이름 : {per}")


In [ ]:
# result_list2 = [[('고려', 'B-ORG'),
#  ('대학교', 'I-ORG'),
#  ('의', 'O'),
#  ('황현수', 'B-PER'),
#  ('는', 'O'),
#  ('안암역', 'B-LOC'),
#  ('에서', 'O'),
#  ('11', 'B-TIM'),
#  ('시', 'I-TIM'),
#  ('30', 'I-TIM'),
#  ('분', 'I-TIM'),
#  ('에', 'O'),
#  ('점심', 'O'),
#  ('을', 'O'),
# ('먹', 'O'),
# ('었', 'O'),
#  ('다', 'O'),
#  ('.', 'O')]]
  
keyword(result_list)    

기관 : 고려대학교 
장소 : 하나스퀘어 
날짜 : 2월9일 
시간 : 11시30분
이름 : 황현수


##음성인식

In [ ]:
!pip install speechrecognition 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
import speech_recognition as sr

sound_file = 'voice.wav'
def sst(sound_file):
  r = sr.Recognizer()
  with sr.AudioFile(sound_file) as source:
      audio = r.record(source)
  text_1 = r.recognize_google(audio, language='ko-KR')
  return text_1


sst(sound_file)

result2:
{   'alternative': [   {   'confidence': 0.73669213,
                           'transcript': '고려대학교 김광현은 부산에서 11시 30분 회의를 진행했다'},
                       {'transcript': '고려대학교 김광현은 11시 30분 회의를 진행했다'},
                       {'transcript': '고려대학교 김강현은 부산에서 11시 30분 회의를 진행했다'},
                       {'transcript': '고려대학교 김광현은 11시 30분에'},
                       {'transcript': '고려대학교 김강현은 11시 30분 회의를 진행했다'}],
    'final': True}


'고려대학교 김광현은 부산에서 11시 30분 회의를 진행했다'

##최종

인풋에 보이스를 넣으면 맞춤법 검사 후 ner 이후 요약까지

In [ ]:
#input
sound_file = 'voice.wav'


def main(sound_file):
  voice2text = sst(sound_file)#음성을 텍스트로
  sent1=text_checker(voice2text)#맞춤법 수정
  result_list = ner_def(sent1)#한문장 넣으면 ner 결과
  keyword(result_list)#결과 프린트

main(sound_file)

result2:
{   'alternative': [   {   'confidence': 0.73669213,
                           'transcript': '고려대학교 김광현은 부산에서 11시 30분 회의를 진행했다'},
                       {'transcript': '고려대학교 김광현은 11시 30분 회의를 진행했다'},
                       {'transcript': '고려대학교 김강현은 부산에서 11시 30분 회의를 진행했다'},
                       {'transcript': '고려대학교 김광현은 11시 30분에'},
                       {'transcript': '고려대학교 김강현은 11시 30분 회의를 진행했다'}],
    'final': True}


100%|██████████| 1/1 [00:00<00:00, 926.30it/s]

1/1 [==============================] - 0s 57ms/step
기관 : 고려대학교 
장소 : 부산 
날짜 :  
시간 : 11시30분
이름 : 김광현


In [ ]:
#음성을 텍스트로
voice2text = sst(sound_file)
voice2text

result2:
{   'alternative': [   {   'confidence': 0.73669237,
                           'transcript': '고려대학교 김광현은 부산에서 11시 30분 회의를 진행했다'},
                       {'transcript': '고려대학교 김광현은 11시 30분 회의를 진행했다'},
                       {'transcript': '고려대학교 김강현은 부산에서 11시 30분 회의를 진행했다'},
                       {'transcript': '고려대학교 김광현은 11시 30분에'},
                       {'transcript': '고려대학교 김강현은 11시 30분 회의를 진행했다'}],
    'final': True}


'고려대학교 김광현은 부산에서 11시 30분 회의를 진행했다'

In [ ]:
#맞춤법 수정
sent1=text_checker(voice2text)
sent1

'고려대학교 김광현은 부산에서 11시 30분 회의를 진행했다'

In [ ]:
#한문장 넣으면 ner 결과
result_list = ner_def(sent1)
result_list

100%|██████████| 1/1 [00:00<00:00, 839.70it/s]

1/1 [==============================] - 0s 73ms/step


[[('고려', 'B-ORG'),
  ('대학교', 'I-ORG'),
  ('김광현', 'B-PER'),
  ('은', 'O'),
  ('부산', 'B-LOC'),
  ('에서', 'O'),
  ('11', 'B-TIM'),
  ('시', 'I-TIM'),
  ('30', 'I-TIM'),
  ('분', 'I-TIM'),
  ('회의', 'O'),
  ('를', 'O'),
  ('진행', 'O'),
  ('했', 'O'),
  ('다', 'O')]]

In [ ]:
#결과 프린트
keyword(result_list)

기관 : 고려대학교 
장소 : 부산 
날짜 :  
시간 : 11시30분
이름 : 김광현
